#### 사용자 정의 모델 클래스
- 부모클래스 : nn.module
- 필수오버라이딩
    * __ init __(): 모델 층 구성 즉, 설계
    * forward(): 순방향 학습 진행 코드 구현

- 동적 모델
    * container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이에 연관성 없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [51]:
# 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

In [52]:
# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 : 동적 모델 <hr>
    - 목표 : 은닉층의 개수가 동적인 모델
    - 조건
        - 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        - 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 퍼셉트론 수

In [53]:
# ----------------------------------------------------------------------------
# 모델 이름  : DynamicModel
# 부모클래스 : nn.Module 
# 매개 변수  : in_in, out_out, h_inout, h_cnt
# ----------------------------------------------------------------------------

In [54]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, out_out, h_inout, h_cnt):
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_inout)
        self.h_layers=nn.ModuleList([nn.Linear(h_inout, h_inout) for _ in range(h_cnt)])
        self.out_layer=nn.Linear(h_inout, out_out)

    # 학습 진행 콜백 메서드
    def forward(self,x):
        # 입력층
        y=self.in_layer(x) # y=x1w1+x2w2+x3w3+b1
        y=F.relu(y)      # 0<=y
        # y=F.relu(self.in_layer(x))
        
        # 은닉층
        for h_layer in self.h_layers:
            y=h_layer(y)
            y=F.relu(y)

        # 출력층
        return self.out_layer(y)

In [55]:
# 모델 인스턴스 생성
ml=DynamicModel(3, 2, 5, 10)

In [56]:
# 모델 구조 확인
print(ml)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (h_layers): ModuleList(
    (0-9): 10 x Linear(in_features=5, out_features=5, bias=True)
  )
  (out_layer): Linear(in_features=5, out_features=2, bias=True)
)


In [57]:
# 모델 파라미터 확인
for name, param in ml.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
h_layers.0.weight torch.Size([5, 5])
h_layers.0.bias torch.Size([5])
h_layers.1.weight torch.Size([5, 5])
h_layers.1.bias torch.Size([5])
h_layers.2.weight torch.Size([5, 5])
h_layers.2.bias torch.Size([5])
h_layers.3.weight torch.Size([5, 5])
h_layers.3.bias torch.Size([5])
h_layers.4.weight torch.Size([5, 5])
h_layers.4.bias torch.Size([5])
h_layers.5.weight torch.Size([5, 5])
h_layers.5.bias torch.Size([5])
h_layers.6.weight torch.Size([5, 5])
h_layers.6.bias torch.Size([5])
h_layers.7.weight torch.Size([5, 5])
h_layers.7.bias torch.Size([5])
h_layers.8.weight torch.Size([5, 5])
h_layers.8.bias torch.Size([5])
h_layers.9.weight torch.Size([5, 5])
h_layers.9.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


- 학습 진행

In [58]:
## 임시 데이터 생성
dataTS=torch.FloatTensor([[1,3,5],[2,4,6],[2,5,7],[3,7,9]]) # 2D (4, 3)
targetTS=torch.FloatTensor([[7,8],[1,2],[5,8],[10,12]])     # 2D (4, 2)

In [59]:
# 모델 학습 진행
pre_y=ml(dataTS)
print(pre_y, pre_y.shape, sep='\n')

tensor([[-0.4371, -0.3432],
        [-0.4371, -0.3432],
        [-0.4370, -0.3432],
        [-0.4370, -0.3432]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
